In [2]:
from cognition_datalab.evaluations import get_workflows, get_workflow_metadata, get_workflow_results

In [3]:
all_workflows = get_workflows()

In [5]:
get_workflow_metadata("Q1-evaluation")

{'workflow': 'Q1-evaluation', 'communications': 56785, 'date': '2023-04-01'}

In [7]:
get_workflow_results("Q1-evaluation")

{'meta': {'workflow': 'Q1-evaluation',
  'communications': 56785,
  'date': '2023-04-01'},
 'classifiers': [{'id': 1,
   'name': 'MODEL:smarsh-secrecy-en:secrecy-en-lr:1.2.0',
   'labels': ['secrecy', 'no']}],
 'communications': [{'messageId': 0,
   'features': [{'recordIdentifier': '0.0',
     'sentence': 'can you please furnish me with your full postal address so that i can send you a copy of the garp brochure in the near future ',
     'classifiers': [{'id': '1',
       'confidence': [0.8384429738447999, 0.1615570261552001]}]},
    {'recordIdentifier': '0.1',
     'sentence': 'she line ',
     'classifiers': [{'id': '1',
       'confidence': [0.7256101928772349, 0.2743898071227651]}]},
    {'recordIdentifier': '0.2',
     'sentence': 'sale wines urltoken ',
     'classifiers': [{'id': '1',
       'confidence': [0.5924834383268096, 0.40751656167319045]}]},
    {'recordIdentifier': '0.3',
     'sentence': 'as pests anyone there ',
     'classifiers': [{'id': '1',
       'confidence': 

In [7]:
get_workflow_results("Q1-evaluation").get("communications")[0]

{'messageId': 0,
 'features': [{'recordIdentifier': '0.0',
   'sentence': 'can you please furnish me with your full postal address so that i can send you a copy of the garp brochure in the near future ',
   'classifiers': [{'id': '1',
     'confidence': [0.8384429738447999, 0.1615570261552001]}]},
  {'recordIdentifier': '0.1',
   'sentence': 'she line ',
   'classifiers': [{'id': '1',
     'confidence': [0.7256101928772349, 0.2743898071227651]}]},
  {'recordIdentifier': '0.2',
   'sentence': 'sale wines urltoken ',
   'classifiers': [{'id': '1',
     'confidence': [0.5924834383268096, 0.40751656167319045]}]},
  {'recordIdentifier': '0.3',
   'sentence': 'as pests anyone there ',
   'classifiers': [{'id': '1',
     'confidence': [0.42186049543776805, 0.578139504562232]}]},
  {'recordIdentifier': '0.4',
   'sentence': 'gary hickerson would need to see this deal due to the cpi and ppi in the pricing ',
   'classifiers': [{'id': '1',
     'confidence': [0.5389376238809899, 0.46106237611901

In [22]:
# calculate the number of positive alerts
THRESHOLD=0.7
def is_alert(communication_features, threshold):
    """given a list of sentence classification, determine if the communication triggered an alert"""
    max_score = max([x[1] for x in [y.get("classifiers")[0].get("confidence") for y in communication_features]])
    if max_score >= threshold:
        return True
    else:
        return False

In [24]:
is_alert(get_workflow_results("Q1-evaluation").get("communications")[1].get("features"), 0.7)

True

In [20]:
get_workflow_results("Q1-evaluation").get("communications")[1]

{'messageId': 1,
 'features': [{'recordIdentifier': '1.0',
   'sentence': 'hoge isnt surprised her adversaries are keeping mum ',
   'classifiers': [{'id': '1',
     'confidence': [0.2577148923140319, 0.7422851076859681]}]},
  {'recordIdentifier': '1.1',
   'sentence': 'a pressing star in zero is not that the school is being recorded ',
   'classifiers': [{'id': '1',
     'confidence': [0.9712537847694555, 0.02874621523054459]}]},
  {'recordIdentifier': '1.2',
   'sentence': 'jean ryall ',
   'classifiers': [{'id': '1',
     'confidence': [0.5334523910238562, 0.4665476089761437]}]},
  {'recordIdentifier': '1.3',
   'sentence': 'uncommontoken trading numtoken numtoken numtoken synth numtoken numtoken numtoken',
   'classifiers': [{'id': '1',
     'confidence': [0.694931984694043, 0.30506801530595695]}]},
  {'recordIdentifier': '1.4',
   'sentence': 'by contrast a calgary canada firm transalta energy averaged moneytoken a megawatt hour and the sacramento municipal utility district had av

In [30]:
%%time
# comm features to flat row
def workflow_to_dataframe(workflow_id, threshold):
    """given a workflow id, return a dataframe of all sentences with their alert status"""
    workflow_results = get_workflow_results(workflow_id)
    all_sentences = []
    for comm in workflow_results.get("communications"):
        message_id = comm.get("messageId")
        alert_status = is_alert(comm.get("features"), threshold)
        for sentence in comm.get("features"):
            all_sentences.append(
                {
                    "message_id": message_id,
                    "sentence_id": sentence.get("recordIdentifier"),
                    "alerted": alert_status, 
                    "sentence": sentence.get("sentence"),
                    "confidence_score": sentence.get("classifiers")[0].get("confidence")[1]
                }
            )
    return pd.DataFrame(all_sentences)

CPU times: user 6 μs, sys: 0 ns, total: 6 μs
Wall time: 8.82 μs


In [34]:
workflow_to_dataframe("Q4-evaluation", 0.7)

,message_id,sentence_id,alerted,sentence,confidence_score
0,0,0.0,False,you can hear them approaching from hundreds of...,0.264727
1,0,0.1,False,so it must be good to work on something other ...,0.362738
2,0,0.2,False,beginning in november i will begin publishing ...,0.499517
3,0,0.3,False,tracey bradley emailtoken to aryeh fishman ema...,0.456301
4,0,0.4,False,you can ride the wayback machine at kahles arc...,0.226483
...,...,...,...,...,...
360206,72042,72042.2,False,she nametoken i think the nyse snow core mount...,0.131291
360207,72042,72042.3,False,an astronomer in an eclipseagain both part deb...,0.598399
360208,72042,72042.4,False,lets try to have the largest group attendance ...,0.144295
360209,72042,72042.5,False,dear dan pressure him defense.,0.486445


In [41]:
df[df.alerted].message_id.nunique()

17069

In [42]:
# get all workflows
all_workflow_ids = get_workflows()
all_metadata = {x: get_workflow_metadata(x) for x in all_workflow_ids}


all_workflow_data = {}
for workflow_id in all_workflow_ids:
    df = workflow_to_dataframe(workflow_id, THRESHOLD)
    all_workflow_data[workflow_id] = df
    all_metadata[workflow_id]["Alerts"] = df[df.alerted].message_id.nunique()

In [50]:
import streamlit as st
for x in zip(all_workflow_ids, st.columns(4)):
    x[1].metric(label = x[0], value = all_metadata[x[0]]["Alerts"])

2024-10-01 11:58:17.506 
  command:

    streamlit run /home/vscode/.local/lib/python3.12/site-packages/ipykernel_launcher.py [ARGUMENTS]
